# The business problem

The fictitous company All Star Jeans is a bran new company that will begin to its e-commerce by selling only jeans for men. However, the company wants to open a new business in selling jeans for men in the United States of America. The company does not know the trends in that market nor about the precification of the products.

Because of this, the company hired us to inform them about pricing, products trends and characteristics of the products in order to be able to compete with other retailers. The main business questions to be answered are:
- **How many different models of products are available at a competitor retailer?**
- **How many different colors are available at other e-commerces?**
- **What are the composition of the material of the clothes?**
- **At what prices the jeans are being sold?**

# Solution

To solve their problem, we are going to use the Beautiful Soup package to web scrap the  website of one of the biggest retailers from the United States, the H&M. To do this, we are going to start a iterative process about understanding the HTML of their website and trying to extract information.

The process is iterative and incremental, and in the next steps we are going to analyze the different steps taken for the solution, such as:

- **Data collection**: accessing the showroom with all available products that we have interest (jeans for men). With this step we are going to extract all available products identifiers;

- **Data collection by product**: with the products identifiers, we can code an algorithm that access the webpage of each product and search for additional information, such as color, pricing, composition;

- **Data cleaning**: after gathering the products and additional information about the product, it is possible that we create duplicated values, columns with null values or even columns that are composed by lists. This step is to clean the data and prepare it to be loaded in a database.

- **Database creation**: in the case of the database not being created, at this time we will create it.

- **Data insertion**: finally, the data acquired and cleaned is inserted in the database. **After the storing of some data, it is possible to see the trends, pricing and materials of products available at the H&M e-commerce**. Thus, offering important insights for the All Star Jeans company.

## Imports

In [19]:
import requests
import math
import re
import pandas as pd
import numpy as np
import logging
import os
import sqlite3

from bs4 import BeautifulSoup
from datetime import datetime
from sqlalchemy import create_engine

In [12]:
# Logging data
path = '/home/pedro/Documentos/repos/web-scraping'

if not os.path.exists(path + 'Logs'):
    os.makedirs(path + 'Logs')

logging.basicConfig(
    filename= path + 'Logs/webscraping_hm' + str(datetime.today()) + '.txt',
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M%S',
    level=logging.INFO
    )
logger = logging.getLogger('webscraping_hm')

## Data collection

Starting the web scrapping process by creating the Beautiful Soup object along with some important parameters for the process.

In [13]:
# Parameters
headers={'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# URL - trying to avoid pagination problem
#url = 'https://www2.hm.com/en_us/men/products/jeans.html'
url = 'https://www2.hm.com/en_us/men/products/jeans.html?sort=stock&image-size=small&image=model&offset=0&page-size=999'

# Request to URL
page = requests.get(url, headers=headers)

# Request response check
print('The page variable will contain the response from the HTML page, where 200 means it worked. The response is: {}'.format(page))

# Beautiful soup object
soup = BeautifulSoup(page.text, 'html.parser')


The page variable will contain the response from the HTML page, where 200 means it worked. The response is: <Response [200]>


With the first test it is possible to see that the object sucessfully connect to the H&M website. The next step for us is to for all of the products (jeans for men) in the showroom. With this project we will be able to collect all the products names and IDs.

In [14]:
# ===================== Product data ===================================#

# Products available from the showroom
products = soup.find('ul', class_='products-listing small')
product_list = products.find_all('article', class_='hm-product-item')

#id
product_id = [p.get('data-articlecode') for p in product_list]
#category
product_cat = [p.get('data-category') for p in product_list]
#product name
name = products.find_all('a', class_='link')
product_name = [p.get_text() for p in name]
#price
price = products.find_all('span', class_='price regular')
product_price = [p.get_text() for p in price]

# Creating the dataframe with the products in it
data = pd.DataFrame([product_name, product_id, product_cat, product_price]).T
data.columns = ['product_name', 'product_id', 'product_cat', 'product_price']

print('The initial scrapping gives to us a dataset of {} size.'.format(data.shape))

The initial scrapping gives to us a dataset of (70, 4) size.


## Data collection by product

With the different product IDs in hand, we can direct the efforts in order to extract more information about each product, such as: colors available, differences in color prices and material used for each pair of jeans.

The main structure for scrapping is by making two different loops,

- External loop: loop around each product ID acquired at the previous step. After this we extract the following information: 
    (i) access the page of the product ID,
    (ii) looks for all colors available for such product ID and
    (iii) gets all the colors IDs available.


- Internal loop: with product ID and color ID, access the page of each product + color ID, looks for its price, material composition and other available information.

In [15]:
# Parameter
headers={'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5), AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

# Empty dataframe
df_compositions = pd.DataFrame()

# Unique columns for all products
aux = []

# Standard format of composition informations
cols = ['Fit', 'Composition', 'Art. No.', 'Product safety', 'Size']
df_pattern = pd.DataFrame(columns=cols)

for i in range( len(data)):
    
    #API Request
    url = 'https://www2.hm.com/en_us/productpage.' + data.loc[i,'product_id'] + '.html'
    page = requests.get( url, headers=headers)
    
    
    #Creating the BeautifulSoup object
    soup = BeautifulSoup( page.text, 'html.parser')
    
    
    #========================================= COLOR =============================================#
    # Starting to search the color name
    product_list = soup.find_all('a', class_='filter-option miniature') + soup.find_all('a', class_='filter-option miniature active')
    colors = [p.get('data-color') for p in product_list]
    
    # Product ID
    product_id = [p.get('data-articlecode') for p in product_list]
    
    # Creating dataframe for product with its color
    df_color = pd.DataFrame( [product_id, colors] ).T
    df_color.columns = ['product_id', 'color']
    
    # Generating style ID + color ID
    #df_color['style_id'] = df_color['product_id'].apply( lambda x: x[:-3])
    #df_color['color_id'] = df_color['product_id'].apply( lambda x: x[-3:])
    
    for j in range(len(df_color)):
        #API Request
        url = 'https://www2.hm.com/en_us/productpage.' + df_color.loc[j,'product_id'] + '.html'
        page = requests.get( url, headers=headers)


        #Creating the BeautifulSoup object
        soup = BeautifulSoup( page.text, 'html.parser')

        #========================================= NAME =============================================#
        product_name_color = soup.find_all('h1', class_='primary product-item-headline')
        product_name_color = product_name_color[0].get_text()
        product_name_color = re.findall(r'\w+\ ?\w+\ ?\w+', product_name_color)[0]
        
        #========================================= PRICE =============================================#
        product_price = soup.find_all('span', class_='price-value') #('div', class_='primary-row product-item-price')
        product_price = product_price[0].get_text()
        product_price = re.findall( r'\S\d+\.?\d+', product_price )[0]
        
        print("URL of the product: {} \n The name of the product is: {}, and its price is: {}".format(url, product_name_color, product_price))

        #========================================= COMPOSITION==========================================#
        # Starting to search product composition
        product_comp = soup.find_all('div', 'pdp-description-list-item')
        composition = [list(filter(None, p.get_text().split('\n') )) for p in product_comp]

        # Creating dataframe
        df_composition = pd.DataFrame(composition).T
        df_composition.columns = df_composition.iloc[0]

        # Deleting the first row of the dataframe
        df_composition = df_composition.iloc[1:].fillna(method='ffill')

        # Removing pocket lining, shell and lining
        df_composition['Composition'] = df_composition['Composition'].str.replace('Pocket lining: ', '', regex=True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Shell: ', '', regex=True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Lining: ', '', regex=True)
        df_composition['Composition'] = df_composition['Composition'].str.replace('Pocket: ', '', regex=True)

        # Guaranteeing the same number of columns
        df_composition = pd.concat([df_pattern, df_composition], axis=0 )

        # Generating style ID + color ID
        df_composition['style_id'] = df_composition['Art. No.'].apply( lambda x: x[:-3])
        df_composition['color_id'] = df_composition['Art. No.'].apply( lambda x: x[-3:])

        # Adding the name and price of the products
        df_composition['product_name'] = product_name_color
        df_composition['product_price'] = product_price
        
        # If a new column appears, it will be attached to the auxiliar list
        aux = aux + df_composition.columns.tolist()

        # Dropping new columns
        df_composition.drop(columns='More sustainable materials', axis='columns', inplace=True, errors='ignore')
        
        # Renaming the columns before the concatenation
        df_composition.columns = ['fit', 'composition', 'product_id','product_safety', 'size', 'style_id', 'color_id', 'product_name', 'product_price']

        # Merging data acquired
        df_composition= pd.merge(df_composition, df_color, how='left', on='product_id')

        # Products with all informations
        df_compositions = pd.concat( [df_compositions, df_composition], axis=0)
        
        
# Scrappy datetime
df_compositions['scrapy_time'] = datetime.now().strftime('%Y-%m-%d %H:%M')

URL of the product: https://www2.hm.com/en_us/productpage.1008549002.html 
 The name of the product is: Regular Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.1008549004.html 
 The name of the product is: Regular Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.1008549006.html 
 The name of the product is: Regular Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.1008549008.html 
 The name of the product is: Regular Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.1008549001.html 
 The name of the product is: Regular Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.0875105001.html 
 The name of the product is: Relaxed Jeans, and its price is: $22.99
URL of the product: https://www2.hm.com/en_us/productpage.0875105002.html 
 The name of the product is: Relaxed Jeans, and its price is:

URL of the product: https://www2.hm.com/en_us/productpage.0875105001.html 
 The name of the product is: Relaxed Jeans, and its price is: $22.99
URL of the product: https://www2.hm.com/en_us/productpage.0875105002.html 
 The name of the product is: Relaxed Jeans, and its price is: $39.99
URL of the product: https://www2.hm.com/en_us/productpage.0875105003.html 
 The name of the product is: Relaxed Jeans, and its price is: $39.99
URL of the product: https://www2.hm.com/en_us/productpage.0875105009.html 
 The name of the product is: Relaxed Jeans, and its price is: $39.99
URL of the product: https://www2.hm.com/en_us/productpage.0875105011.html 
 The name of the product is: Relaxed Jeans, and its price is: $22.99
URL of the product: https://www2.hm.com/en_us/productpage.0875105015.html 
 The name of the product is: Relaxed Jeans, and its price is: $22.99
URL of the product: https://www2.hm.com/en_us/productpage.0875105017.html 
 The name of the product is: Relaxed Jeans, and its price is:

URL of the product: https://www2.hm.com/en_us/productpage.0985159001.html 
 The name of the product is: Skinny Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.0985159002.html 
 The name of the product is: Skinny Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.0985159003.html 
 The name of the product is: Skinny Jeans, and its price is: $11.99
URL of the product: https://www2.hm.com/en_us/productpage.0985159005.html 
 The name of the product is: Skinny Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.0985159006.html 
 The name of the product is: Skinny Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.0985159007.html 
 The name of the product is: Skinny Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.0985159008.html 
 The name of the product is: Skinny Jeans, and its price is: $19.99

URL of the product: https://www2.hm.com/en_us/productpage.1004199001.html 
 The name of the product is: Skinny Cropped Jeans, and its price is: $29.99
URL of the product: https://www2.hm.com/en_us/productpage.1004199002.html 
 The name of the product is: Skinny Cropped Jeans, and its price is: $29.99
URL of the product: https://www2.hm.com/en_us/productpage.1004199003.html 
 The name of the product is: Skinny Cropped Jeans, and its price is: $29.99
URL of the product: https://www2.hm.com/en_us/productpage.1004199005.html 
 The name of the product is: Skinny Cropped Jeans, and its price is: $29.99
URL of the product: https://www2.hm.com/en_us/productpage.1024256001.html 
 The name of the product is: Slim Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.1024256002.html 
 The name of the product is: Slim Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.1024256003.html 
 The name of the product is: Slim Jeans

URL of the product: https://www2.hm.com/en_us/productpage.1013317001.html 
 The name of the product is: Hybrid Regular Tapered, and its price is: $39.99
URL of the product: https://www2.hm.com/en_us/productpage.0985159001.html 
 The name of the product is: Skinny Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.0985159002.html 
 The name of the product is: Skinny Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.0985159003.html 
 The name of the product is: Skinny Jeans, and its price is: $11.99
URL of the product: https://www2.hm.com/en_us/productpage.0985159004.html 
 The name of the product is: Skinny Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.0985159005.html 
 The name of the product is: Skinny Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.0985159006.html 
 The name of the product is: Skinny Jeans, and its price 

URL of the product: https://www2.hm.com/en_us/productpage.1024256003.html 
 The name of the product is: Slim Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.1024256004.html 
 The name of the product is: Slim Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.1024256006.html 
 The name of the product is: Slim Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.1024256007.html 
 The name of the product is: Slim Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.1024256005.html 
 The name of the product is: Slim Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.0985159001.html 
 The name of the product is: Skinny Jeans, and its price is: $19.99
URL of the product: https://www2.hm.com/en_us/productpage.0985159002.html 
 The name of the product is: Skinny Jeans, and its price is: $19.99
URL of th

URL of the product: https://www2.hm.com/en_us/productpage.1004199003.html 
 The name of the product is: Skinny Cropped Jeans, and its price is: $29.99
URL of the product: https://www2.hm.com/en_us/productpage.1004199005.html 
 The name of the product is: Skinny Cropped Jeans, and its price is: $29.99
URL of the product: https://www2.hm.com/en_us/productpage.1004199001.html 
 The name of the product is: Skinny Cropped Jeans, and its price is: $29.99
URL of the product: https://www2.hm.com/en_us/productpage.1013317001.html 
 The name of the product is: Hybrid Regular Tapered, and its price is: $39.99
URL of the product: https://www2.hm.com/en_us/productpage.1013317002.html 
 The name of the product is: Hybrid Regular Tapered, and its price is: $39.99
URL of the product: https://www2.hm.com/en_us/productpage.1013317004.html 
 The name of the product is: Hybrid Regular Tapered, and its price is: $20.99
URL of the product: https://www2.hm.com/en_us/productpage.1013317005.html 
 The name of 

URL of the product: https://www2.hm.com/en_us/productpage.0875105016.html 
 The name of the product is: Relaxed Jeans, and its price is: $39.99
URL of the product: https://www2.hm.com/en_us/productpage.0875105017.html 
 The name of the product is: Relaxed Jeans, and its price is: $39.99
URL of the product: https://www2.hm.com/en_us/productpage.0875105018.html 
 The name of the product is: Relaxed Jeans, and its price is: $39.99
URL of the product: https://www2.hm.com/en_us/productpage.0875105023.html 
 The name of the product is: Relaxed Jeans, and its price is: $39.99
URL of the product: https://www2.hm.com/en_us/productpage.0875105024.html 
 The name of the product is: Relaxed Jeans, and its price is: $39.99
URL of the product: https://www2.hm.com/en_us/productpage.0938875001.html 
 The name of the product is: Slim Tapered Jeans, and its price is: $29.99
URL of the product: https://www2.hm.com/en_us/productpage.0938875002.html 
 The name of the product is: Slim Tapered Jeans, and its

URL of the product: https://www2.hm.com/en_us/productpage.0427159002.html 
 The name of the product is: Trashed Skinny Jeans, and its price is: $39.99
URL of the product: https://www2.hm.com/en_us/productpage.0427159003.html 
 The name of the product is: Trashed Skinny Jeans, and its price is: $39.99
URL of the product: https://www2.hm.com/en_us/productpage.0427159004.html 
 The name of the product is: Trashed Skinny Jeans, and its price is: $39.99
URL of the product: https://www2.hm.com/en_us/productpage.0427159005.html 
 The name of the product is: Trashed Skinny Jeans, and its price is: $39.99
URL of the product: https://www2.hm.com/en_us/productpage.0427159007.html 
 The name of the product is: Trashed Skinny Jeans, and its price is: $18.99
URL of the product: https://www2.hm.com/en_us/productpage.0427159008.html 
 The name of the product is: Trashed Skinny Jeans, and its price is: $18.99
URL of the product: https://www2.hm.com/en_us/productpage.0427159010.html 
 The name of the pr

URL of the product: https://www2.hm.com/en_us/productpage.1004476002.html 
 The name of the product is: Freefit, and its price is: $29.99
URL of the product: https://www2.hm.com/en_us/productpage.1004476003.html 
 The name of the product is: Freefit, and its price is: $24.99
URL of the product: https://www2.hm.com/en_us/productpage.1004476004.html 
 The name of the product is: Freefit, and its price is: $28.99
URL of the product: https://www2.hm.com/en_us/productpage.1004476005.html 
 The name of the product is: Freefit, and its price is: $34.99
URL of the product: https://www2.hm.com/en_us/productpage.1004476006.html 
 The name of the product is: Freefit, and its price is: $49.99
URL of the product: https://www2.hm.com/en_us/productpage.0974202001.html 
 The name of the product is: Regular Denim Joggers, and its price is: $17.99
URL of the product: https://www2.hm.com/en_us/productpage.0974202003.html 
 The name of the product is: Regular Denim Joggers, and its price is: $29.99
URL of

In [5]:
## Joining showroom data + details
#data['style_id'] = data['product_id'].apply( lambda x: x[:-3])
#data['color_id'] = data['product_id'].apply( lambda x: x[-3:])

#data_raw = pd.merge(data, df_details[['style_id', 'color', 'Fit', 'Composition', 'Size', 'Product safety']], how='left', on='style_id')

# Checking dataframe size before exporting to CSV
#print("The dimensions of the data frame are: {}".format(data_raw.shape))

# Saving a copy of the raw data after scrapping
#data_raw.to_csv('datasets/data_raw_hm.csv', index=False)

## Data cleaning

Now, every product available in every color has been properly extract from the web. However, during the process some duplicates may have been added to the dataframe, also there are some columns with nested attributes, such as composition. Our next steps is to clean the data by finding and dropping duplicates, treat null values and decompose the composition column into new columns, each representing the percentage of a material.

In [16]:
# Loading raw data from previous step or from a CSV
data = df_compositions.copy() # data = pd.read_csv('datasets/data_raw_hm.csv')

print('Before cleaning, the dataset has the dimensions of: {}, with {} unique products'.format(data.shape, len(data['product_id'].unique())))

# product_id
data = data.dropna(subset=['product_id'])
#data['product_id'] = data['product_id'].astype(int)

# product_category

# product_name
data['product_name'] = data['product_name'].apply(lambda x: x.replace(' ', '_').lower() if pd.notnull(x) else x)

# Removing $ from the product price
data['product_price'] = data['product_price'].apply(lambda x: x.replace('$', '') if pd.notnull(x) else x )
# product_price
data['product_price'] = data['product_price'].astype(float)

# scrapy datetime
data['scrapy_time'] = pd.to_datetime( data['scrapy_time'], format='%Y-%m-%d %H:%M:%S' )

# style_id
#data['style_id'] = data['style_id'].astype(int)

# color_id
#data['color_id'] = data['color_id'].astype(int)

# color
data['color'] = data['color'].apply(lambda x: x.replace(' ', '_').replace('/','_').lower() if pd.notnull(x) else x)

# fit
data['fit'] = data['fit'].apply(lambda x: x.replace(' ', '_').replace('/','_').lower() if pd.notnull(x) else x)


# size - from size, we will extract 2 features with regex
data['size_number'] = data['size'].apply(lambda x: re.search('\d{3}cm', x).group(0)  if pd.notnull(x) else x)
data['size_number'] = data['size_number'].apply(lambda x: x.replace('cm', '') if pd.notnull(x) else x)

data['size_model'] = data['size'].str.extract( '(\d+/\\d+)' )

# removing duplicates
#data = data.drop_duplicates(subset=['product_name', 'product_id', 'product_cat', 'product_price',
#       'scrapy_time', 'style_id', 'color_id', 'color', 'Fit'], keep='last')

# reset index
#data = data.reset_index(drop=True)

# ==================================== COMPOSITION =======================================
# break composition
df_comp = data['composition'].str.split(',', expand=True).reset_index(drop=True)
df_ref = pd.DataFrame(index=np.arange( len(data)), columns=['cotton','polyester','elastane', 'elasterell','other'])

# looking for cotton composition in df_comp -------------------- Cotton ---------------
df_cotton_0 = df_comp.loc[df_comp[0].str.contains('Cotton', na=True), 0]
df_cotton_0.name = 'cotton'
df_cotton_1 = df_comp.loc[df_comp[1].str.contains('Cotton', na=True), 1]
df_cotton_1.name = 'cotton'
# combining cotton results
df_cotton = df_cotton_0.combine_first(df_cotton_1)
df_ref = pd.concat([df_ref,df_cotton], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# looking for polyester composition in df_comp -------------------- Polyester ---------------
df_polyester_0 = df_comp.loc[df_comp[0].str.contains('Polyester', na=True), 0]
df_polyester_0.name = 'polyester'
df_polyester_1 = df_comp.loc[df_comp[1].str.contains('Polyester', na=True), 1]
df_polyester_1.name = 'polyester'
# combining polyester results
df_polyester = df_polyester_0.combine_first(df_polyester_1)
df_ref = pd.concat([df_ref,df_polyester], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# looking for elastane composition in df_comp -------------------- Elastane ---------------
df_elastane_1 = df_comp.loc[df_comp[0].str.contains('Spandex', na=True), 0]
df_elastane_1.name = 'spandex'
df_elastane_2 = df_comp.loc[df_comp[1].str.contains('Spandex', na=True), 1]
df_elastane_2.name = 'spandex'
df_elastane_3 = df_comp.loc[df_comp[2].str.contains('Spandex', na=True), 2]
df_elastane_3.name = 'spandex'
# combining elastane results
df_elastane_interm = df_elastane_1.combine_first(df_elastane_2)
df_elastane = df_elastane_interm.combine_first(df_elastane_3)
df_ref = pd.concat([df_ref,df_elastane], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# looking for elasterell composition in df_comp -------------------- Elasterell ---------------
df_elasterell_1 = df_comp.loc[df_comp[1].str.contains('Elasterell', na=True), 1]
df_elasterell_1.name = 'elasterell'
# combining elasterell results
df_ref = pd.concat([df_ref,df_elasterell_1], axis=1)
df_ref = df_ref.iloc[:, ~df_ref.columns.duplicated(keep='last')]

# joining with the product_id
df_aux = pd.concat([data['product_id'].reset_index(drop=True), df_ref], axis=1)

# format composition data
df_aux['cotton'] = df_aux['cotton'].apply(lambda x: int(re.search('\d+', x).group(0)) if pd.notnull(x) else 0)
df_aux['polyester'] = df_aux['polyester'].apply(lambda x: int(re.search('\d+', x).group(0)) if pd.notnull(x) else 0)
df_aux['spandex'] = df_aux['spandex'].apply(lambda x: int(re.search('\d+', x).group(0)) if pd.notnull(x) else 0)
df_aux['elasterell'] = df_aux['elasterell'].apply(lambda x: int(re.search('\d+', x).group(0)) if pd.notnull(x) else 0)
#df_aux['other'] = df_aux['other'].apply(lambda x: int(re.search('\d+', x).group(0)) if pd.notnull(x) else 0)

# final join
df_aux = df_aux.groupby('product_id').max().reset_index().fillna(0)
data = pd.merge(data, df_aux, on='product_id', how='left')

# droping unnecessary data
data = data.drop(columns=['size', 'product_safety', 'composition', 'other'])

# Drop duplicates
data = data.drop_duplicates()

# Checking dataframe size after cleaning 
print("The dimensions of the data frame after cleaning is: {}".format(data.shape))

Before cleaning, the dataset has the dimensions of: (938, 11), with 188 unique products
The dimensions of the data frame after cleaning is: (188, 15)


## Creating a database to store results

With all the data cleaned, we can proceed to create the database that will store our results from the web scrapping process. The database is going to be a relational database based on the SQLite engine.

The main objective of the database is to keep track of all products available at H&M e-commerce for men jeans, in order to provide insights for the company about prices, colors available, number of products along different periods of time.

In [25]:
query_showroom_schema = '''
    CREATE TABLE vitrine(
    product_id     TEXT,
    style_id       TEXT,
    color_id       TEXT,
    product_name   TEXT,
    color          TEXT,
    fit            TEXT,
    product_price  REAL,
    size_number    TEXT,
    size_model     TEXT,
    cotton         REAL,
    polyester      REAL,
    spandex       REAL,
    elasterell     REAL,
    scrapy_time    TEXT    
    )
'''

# Droping the table - NOT TO USE
#query_drop = '''
#    DROP TABLE vitrine
#'''


# Create table
try:
    conn = sqlite3.connect('database_hm.sqlite')
    cursor = conn.execute(query_showroom_schema)
    conn.commit()
    #conn.exit()
except:
    print('Database already exists')


Database already exists


## Loading data into the database

Now, the last step is to insert the data acquired and cleaned into the database.

In [30]:
# Inserting data into the database
try:
    data_insert = data[[
    'product_id',
    'style_id',
    'color_id',
    'product_name',
    'color',
    'fit',
    'product_price',
    'size_number',
    'size_model',
    'cotton',
    'polyester',
    'spandex',
    'elasterell',
    'scrapy_time']]
        
    # Creating connection to database
    conn = create_engine('sqlite:///database_hm.sqlite', echo=False)

    # Inserting data
    data_insert.to_sql('vitrine', con=conn, if_exists='append', index=False)
except:
    print('Some error occurred and data was not inserted')

## Next steps

The next steps for this project is to analyze all the data saved and the trends at H&M e-commerce website. For this, we are using the Cron job scheduler to run the web scrapping script every week until enough data for the analysis is collected.